In [1]:
import pandas as pd
import pickle
import numpy as np

file_path = "../data/ref_data.csv"
SCALER_PATH = "../artifacts/scaler.pkl"
MODEL_PATH = "../artifacts/model.pkl"

# Map des émotions prédictibles
EMOTION_MAP = {
    0: "neutral",
    1: "calm",
    2: "happy",
    3: "sad",
    4: "angry",
    5: "fearful",
    6: "disgust",
    7: "surprised"
}


In [5]:
# 1) Lecture
data = pd.read_csv(file_path)

# 2) Renommer la colonne 'true_label' en 'target'
#    et 'true_label_encoded' en 'target_encoded'
data.rename(
    columns={
        "true_label": "target",
        "true_label_encoded": "target_encoded"
    },
    inplace=True
)

data

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_1257,feature_1258,feature_1259,feature_1260,feature_1261,feature_1262,feature_1263,feature_1264,target,target_encoded
0,-726.21720,68.541420,3.293398,12.205300,5.510278,13.667408,-2.983828,3.098029,-3.310813,-1.564384,...,0.000283,0.000310,0.000303,0.000316,0.000304,0.000274,0.000288,0.000286,neutral,5
1,-719.12830,70.201570,1.168397,13.122541,7.836950,14.411290,-4.111360,4.468973,-3.539367,-3.658608,...,0.000306,0.000310,0.000306,0.000302,0.000300,0.000285,0.000275,0.000242,neutral,5
2,-714.99570,69.689350,3.924564,11.924190,6.421723,11.011614,-2.878103,4.509558,-4.476109,-2.671549,...,0.000255,0.000259,0.000288,0.000284,0.000284,0.000261,0.000258,0.000254,neutral,5
3,-710.97530,67.564880,5.782241,13.230726,6.190846,12.628252,-1.675169,5.657494,-4.950634,-3.477546,...,0.000283,0.000289,0.000293,0.000280,0.000279,0.000285,0.000272,0.000221,neutral,5
4,-759.92175,75.783520,6.023605,14.557394,6.454187,14.631508,-3.004551,4.620970,-5.200016,-0.707430,...,0.000268,0.000264,0.000277,0.000279,0.000279,0.000293,0.000264,0.000221,calm,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,-616.32245,45.899370,-20.175966,3.753582,-11.131477,-3.964839,-14.135321,-4.771223,-15.621428,-8.477671,...,0.000314,0.000319,0.000297,0.000301,0.000312,0.000313,0.000311,0.000270,surprised,7
1436,-553.99200,56.614850,-14.907667,2.665701,-13.776782,-4.072508,-17.319796,-5.079650,-10.142247,-8.829944,...,0.000343,0.000341,0.000336,0.000337,0.000330,0.000355,0.000367,0.000337,surprised,7
1437,-575.84576,53.281605,-21.947104,5.056113,-12.553356,-2.494527,-20.614610,-6.246452,-10.795346,-8.800463,...,0.000350,0.000355,0.000356,0.000342,0.000338,0.000367,0.000371,0.000301,surprised,7
1438,-522.46700,43.700798,-14.606612,12.619474,-11.496836,1.743054,-13.268276,0.747497,-13.840753,-2.510027,...,0.000418,0.000433,0.000398,0.000364,0.000353,0.000398,0.000408,0.000330,surprised,7


In [6]:
with open(SCALER_PATH, 'rb') as f:
    scaler = pickle.load(f)

with open(MODEL_PATH, 'rb') as f:
    model = pickle.load(f)

C:\Users\Heidi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
# Identifie les colonnes de features
feature_cols = [col for col in data.columns if col.startswith("feature_")]

# Extraction de la matrice X
X = data[feature_cols].values  # shape (n_samples, 1265) par exemple

# Applique le scaler
X_scaled = scaler.transform(X)


C:\Users\Heidi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [14]:
y_pred_numeric = np.argmax(model.predict(X_scaled), axis=1)
y_pred_numeric

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([5, 5, 5, ..., 4, 7, 7], dtype=int64)

In [17]:
data["prediction_encoded"] = y_pred_numeric

In [18]:
data["prediction"] = data["prediction_encoded"].map(EMOTION_MAP)

In [19]:
data

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_1259,feature_1260,feature_1261,feature_1262,feature_1263,feature_1264,target,target_encoded,prediction,prediction_encoded
0,-726.21720,68.541420,3.293398,12.205300,5.510278,13.667408,-2.983828,3.098029,-3.310813,-1.564384,...,0.000303,0.000316,0.000304,0.000274,0.000288,0.000286,neutral,5,fearful,5
1,-719.12830,70.201570,1.168397,13.122541,7.836950,14.411290,-4.111360,4.468973,-3.539367,-3.658608,...,0.000306,0.000302,0.000300,0.000285,0.000275,0.000242,neutral,5,fearful,5
2,-714.99570,69.689350,3.924564,11.924190,6.421723,11.011614,-2.878103,4.509558,-4.476109,-2.671549,...,0.000288,0.000284,0.000284,0.000261,0.000258,0.000254,neutral,5,fearful,5
3,-710.97530,67.564880,5.782241,13.230726,6.190846,12.628252,-1.675169,5.657494,-4.950634,-3.477546,...,0.000293,0.000280,0.000279,0.000285,0.000272,0.000221,neutral,5,fearful,5
4,-759.92175,75.783520,6.023605,14.557394,6.454187,14.631508,-3.004551,4.620970,-5.200016,-0.707430,...,0.000277,0.000279,0.000279,0.000293,0.000264,0.000221,calm,1,calm,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,-616.32245,45.899370,-20.175966,3.753582,-11.131477,-3.964839,-14.135321,-4.771223,-15.621428,-8.477671,...,0.000297,0.000301,0.000312,0.000313,0.000311,0.000270,surprised,7,surprised,7
1436,-553.99200,56.614850,-14.907667,2.665701,-13.776782,-4.072508,-17.319796,-5.079650,-10.142247,-8.829944,...,0.000336,0.000337,0.000330,0.000355,0.000367,0.000337,surprised,7,surprised,7
1437,-575.84576,53.281605,-21.947104,5.056113,-12.553356,-2.494527,-20.614610,-6.246452,-10.795346,-8.800463,...,0.000356,0.000342,0.000338,0.000367,0.000371,0.000301,surprised,7,angry,4
1438,-522.46700,43.700798,-14.606612,12.619474,-11.496836,1.743054,-13.268276,0.747497,-13.840753,-2.510027,...,0.000398,0.000364,0.000353,0.000398,0.000408,0.000330,surprised,7,surprised,7


In [20]:
data.to_csv("../data/ref_data_with_prediction.csv", index=False)